# Bagging and boosting on properties dataset of Zillow

In today's precept, we are going to work on the Zillow dataset again and apply what we saw in class about:
- Bagging
- Boosting

## Bagging, boosting and random forests

Bagging and boosting are two types of ensemble learning. They decrease the variance of a single estimate as they combine several estimates from different models. So the result may be a model with higher stability.

- Bagging: It is a *homogeneous* weak learners’ model that learns from each other independently in parallel and combines them for determining the model average.
- Random forests: In Bagging, the overlap in the bootstrapped samples causes the outcomes to be highly correlated. To solve this, the idea is to decorrelate the outputs by limiting the dimension of the feature space at each bootstrap sampling ($p \choose m$, $m \approx \sqrt p$ for classification, $m \approx p/3$ for regression.
- Boosting: It is also a *homogeneous* weak learners’ model but works differently from Bagging. In this model, learners learn sequentially and adaptively to improve model predictions of a learning algorithm.


### Bagging

Boostrap (sampling with repetition) aggregating. Special case of the model averaging approach.

![Bagging](Bagging.png)

## Boosting
Boosting is a way of fitting an additive expansion in a set of elementary “basis” functions. It is done by building a model by using weak models in series. Firstly, a model is built from the training data. Then the second model is built which tries to correct the errors (residuals / missclassifications) present in the first model. This procedure is continued and models are added until either the complete training data set is predicted correctly or the maximum number of models is added.

Recall:

$\hat{y}_i = F_M(x_i) = \sum_{m=1}^{M} h_m(x_i)$ where the $h_m$ are estimators called weak learners in the context of boosting. In boosting algorithms, $\hat{y}_i$ is built in a greedy fashion. 

$F_m(x) = F_{m-1}(x) + h_m(x)$ with $h_m =  \arg\min_{h} L_m = \arg\min_{h} \sum_{i=1}^{n} l(y_i, F_{m-1}(x_i) + h(x_i))$, l being a given loss function.

$l(y_i, F_{m-1}(x_i) + h_m(x_i)) \approx l(y_i, F_{m-1}(x_i)) + h_m(x_i) \left[ \frac{\partial l(y_i, F(x_i))}{\partial F(x_i)} \right]_{F=F_{m - 1}}$

$h_m \approx \arg\min_{h} \sum_{i=1}^{n} h(x_i) \left[ \frac{\partial l(y_i, F(x_i))}{\partial F(x_i)} \right]_{F=F_{m - 1}} \geq -||h(x)||^2.||\nabla_F l||_{F=F_{m - 1}}^2$

with equality i.i.f. $h(x) \propto -\nabla_F l_{F=F_{m - 1}}$ by Cauchy-Schwarz. Therefore, at each iteration, the estimator $h_m$ is **fitted to predict the negative gradients of the samples**.

### AdaBoost

![AdaBoost](Boosting.png)

$$L(y, f(x)) = exp(-yf(x))$$

For AdaBoost the basis functions are the individual classifiers $G_m(x) \in \{−1,1\}$. 

Let $(x_1, y_1), ..., (x_n, y_n)$ be n data points, $y_i \in \{-1, 1\}$.

$$ (\beta_m, G_m) \in argmin \sum_{i=1}^n exp(-y_i(f_{m-1}(x_i) + \beta_mG_m(x_i)) [1]$$



We can set $w_i = exp(-y_if_{m-1}(x_i))$ to see that, at iteration m, we are fitting the model on reweighted data. 

We update: $f_m(x) = f_{m-1}(x) + \beta_m G_m(x))$.

See section 10.4, *The Elements of Statistical Learning*, Tibshirani & al., for further details on how to solve [1].

### Data analysis

In [192]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# Set Matplotlib defaults
plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True, figsize=(11, 5))
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=14,
    titlepad=10,
)
plot_params = dict(
    color="0.75",
    style=".-",
    markeredgecolor="0.25",
    markerfacecolor="0.25",
    legend=False,
)
%config InlineBackend.figure_format = 'retina'
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.linear_model import enet_path, ElasticNet, LinearRegression, Ridge, Lasso
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split, KFold
from sklearn import metrics
import math
from collections import Counter
import random
from scipy.stats import multivariate_normal
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy import stats
from tqdm import tqdm
from numba import jit, njit
from sklearn.ensemble import BaggingRegressor, GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import KFold

## Read the data

In [193]:
train = pd.read_csv("train.data.csv", low_memory = False, index_col = 0)
test = pd.read_csv("test.data.csv", low_memory = False, index_col = 0)

In [194]:
train.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
2,6414100192,20141209T000000,538000,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
4,2487200875,20141209T000000,604000,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
5,1954400510,20150218T000000,510000,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
6,7237550310,20140512T000000,1225000,4,4.50,5420,101930,1.0,0,0,...,11,3890,1530,2001,0,98053,47.6561,-122.005,4760,101930
7,1321400060,20140627T000000,257500,3,2.25,1715,6819,2.0,0,0,...,7,1715,0,1995,0,98003,47.3097,-122.327,2238,6819


In [195]:
train.shape

(15129, 21)

In [196]:
train.dropna(inplace = True)
test.dropna(inplace = True)

In [197]:
train.shape

(15129, 21)

As before, we add the OHE "zipcode".

In [198]:
ohe = OneHotEncoder()
categories_train = ohe.fit_transform(train.loc[:, ['zipcode']]).toarray()

In [199]:
ohe.categories_[0]

array([98001, 98002, 98003, 98004, 98005, 98006, 98007, 98008, 98010,
       98011, 98014, 98019, 98022, 98023, 98024, 98027, 98028, 98029,
       98030, 98031, 98032, 98033, 98034, 98038, 98039, 98040, 98042,
       98045, 98052, 98053, 98055, 98056, 98058, 98059, 98065, 98070,
       98072, 98074, 98075, 98077, 98092, 98102, 98103, 98105, 98106,
       98107, 98108, 98109, 98112, 98115, 98116, 98117, 98118, 98119,
       98122, 98125, 98126, 98133, 98136, 98144, 98146, 98148, 98155,
       98166, 98168, 98177, 98178, 98188, 98198, 98199])

In [200]:
categories_test = ohe.transform(test.loc[:, ['zipcode']]).toarray()

In [201]:
for i, cat in enumerate(ohe.categories_[0]):
    train[str(cat)] = categories_train[:, i]
    test[str(cat)] = categories_test[:, i]

In [202]:
train.drop(['zipcode'], axis = 1, inplace = True)
test.drop(['zipcode'], axis = 1, inplace = True)

In [203]:
train.columns.unique()

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'lat',
       'long', 'sqft_living15', 'sqft_lot15', '98001', '98002', '98003',
       '98004', '98005', '98006', '98007', '98008', '98010', '98011', '98014',
       '98019', '98022', '98023', '98024', '98027', '98028', '98029', '98030',
       '98031', '98032', '98033', '98034', '98038', '98039', '98040', '98042',
       '98045', '98052', '98053', '98055', '98056', '98058', '98059', '98065',
       '98070', '98072', '98074', '98075', '98077', '98092', '98102', '98103',
       '98105', '98106', '98107', '98108', '98109', '98112', '98115', '98116',
       '98117', '98118', '98119', '98122', '98125', '98126', '98133', '98136',
       '98144', '98146', '98148', '98155', '98166', '98168', '98177', '98178',
       '98188', '98198', '98199'],
      dtype='object')

In [204]:
test.shape

(6484, 90)

In [205]:
train.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,98146,98148,98155,98166,98168,98177,98178,98188,98198,98199
count,1.512900e+04,1.512900e+04,15129.000000,15129.000000,15129.000000,1.512900e+04,15129.000000,15129.000000,15129.000000,15129.000000,...,15129.000000,15129.000000,15129.000000,15129.000000,15129.000000,15129.000000,15129.000000,15129.000000,15129.000000,15129.000000
mean,4.561012e+09,5.381469e+05,3.365854,2.107327,2069.595743,1.469552e+04,1.490878,0.007403,0.234649,3.409545,...,0.012625,0.002776,0.020953,0.011369,0.012757,0.011435,0.011898,0.006081,0.013087,0.015467
std,2.868152e+09,3.674490e+05,0.938624,0.772123,918.759850,3.887165e+04,0.539352,0.085724,0.769640,0.651771,...,0.111652,0.052618,0.143232,0.106021,0.112228,0.106325,0.108429,0.077746,0.113653,0.123405
min,1.000102e+06,8.100000e+04,0.000000,0.000000,290.000000,6.000000e+02,1.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.121000e+09,3.220000e+05,3.000000,1.500000,1420.000000,5.030000e+03,1.000000,0.000000,0.000000,3.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.885808e+09,4.500000e+05,3.000000,2.250000,1900.000000,7.577000e+03,1.500000,0.000000,0.000000,3.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,7.299500e+09,6.400000e+05,4.000000,2.500000,2540.000000,1.060900e+04,2.000000,0.000000,0.000000,4.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,9.900000e+09,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [206]:
train.drop(['id', 'date'], axis = 1, inplace = True)
test.drop(['id', 'date'], axis = 1, inplace = True)

In [207]:
X_train, y_train = train.drop(['price'], axis=1), train['price']
X_test, y_test = test.drop(['price'], axis=1), test['price']

In [208]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.4, random_state=200)

In [209]:
X_train.iloc[:, :18].head()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,lat,long,sqft_living15,sqft_lot15,98001
2867,3,2.25,1630,10500,1.0,0,0,3,7,1100,530,1974,0,47.7176,-122.236,1640,9794,0.0
16875,4,2.50,2280,9827,2.0,0,0,3,8,2280,0,1995,0,47.6883,-122.168,1660,9827,0.0
3202,2,1.00,1790,4000,1.0,0,0,4,7,1040,750,1923,0,47.6405,-122.301,1310,4000,0.0
19207,4,2.75,2890,6825,1.0,0,0,3,8,1560,1330,1973,0,47.7340,-122.182,1900,10120,0.0
13020,3,1.00,1010,6300,1.0,0,0,3,7,1010,0,1957,0,47.7304,-122.352,1480,7480,0.0


In [210]:
for i, c in enumerate(X_train.columns):
    if i<18 and c!='view' and c!='waterfront':
        mu, sigma = np.mean(X_train.loc[:, c]), np.std(X_train.loc[:, c])
        X_train.loc[:, c] = (X_train.loc[:, c]-mu)/sigma
        X_val.loc[:, c] = (X_val.loc[:, c]-mu)/sigma
        X_test.loc[:, c] = (X_test.loc[:, c]-mu)/sigma
mu_y_train = np.mean(y_train)
sigma_y_train = np.std(y_train)
y_train = (y_train - mu_y_train)/sigma_y_train

In [211]:
X_train.head()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,...,98146,98148,98155,98166,98168,98177,98178,98188,98198,98199
2867,-0.378446,0.195006,-0.472042,-0.115888,-0.914479,0,0,-0.630641,-0.555241,-0.820235,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16875,0.674313,0.521767,0.239355,-0.134220,0.923591,0,0,-0.630641,0.297971,0.607077,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3202,-1.431204,-1.438800,-0.296929,-0.292945,-0.914479,0,0,0.901156,-0.555241,-0.892810,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19207,0.674313,0.848528,0.906975,-0.215994,-0.914479,0,0,-0.630641,0.297971,-0.263825,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13020,-0.378446,-1.438800,-1.150606,-0.230294,-0.914479,0,0,-0.630641,-0.555241,-0.929098,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [212]:
lr = LinearRegression()
lr.fit(X_train,y_train)

LinearRegression()

In [213]:
predictions = lr.predict(X_test)*sigma_y_train + mu_y_train

In [214]:
lr.score(X_train, y_train)

0.806439622281405

In [215]:
R2_oos = 1 - np.mean((predictions - y_test)**2)/np.var(y_test)

In [216]:
R2_oos

0.7977665204038027

## Adding interaction terms

In [217]:
interactions = sorted(['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot'])

In [218]:
for c1 in interactions:
    for c2 in interactions:
        if c1<c2:
            # we add first order intercation term here
            X_train[f"{c1}x{c2}"] = X_train.loc[:, c1]*X_train.loc[:, c2]
            X_val[f"{c1}x{c2}"] = X_val.loc[:, c1]*X_val.loc[:, c2]
            X_test[f"{c1}x{c2}"] = X_test.loc[:, c1]*X_test.loc[:, c2]

In [219]:
X_train.shape

(9077, 93)

In [220]:
X_val.shape

(6052, 93)

In [221]:
lr = LinearRegression()

In [222]:
lr.fit(X_train,y_train)

LinearRegression()

In [223]:
predictions = lr.predict(X_test)*sigma_y_train + mu_y_train

In [224]:
lr.score(X_train, y_train)

0.8482355295948942

In [225]:
R2_oos = 1 - np.mean((predictions - y_test)**2)/np.var(y_test)

In [226]:
R2_oos

0.8267179989435924

## Adding splines

Let $t_0 = a, t_1, t_2, ..., t_n = b$ be a partition of $[a, b]$. A spline function B of degree d on $[a, b]$ is a piecewise polynomial function of degree d on intervals $[t_i, t_{i+1}] (i \in [0, n])$ with continuous d-1 derivatives. The knots of B are the points $t_1, ... t_{n-1}$ where the spline might not have continuous $d^{th}$ derivatives.

In this example, we are going to add quadratic splines with 9 knots which are the *10.i* deciles of the *sqft_living* variable for $i \in [1, ..., 9]$.

In [227]:
deciles = np.percentile(X_train.sqft_living, [k*10 for k in range(0, 10)])

In [228]:
for i, tau in enumerate(deciles):
    
    temp_train = X_train.sqft_living - tau
    temp_train[temp_train<0] = 0
    
    temp_val = X_val.sqft_living - tau
    temp_val[temp_val<0] = 0
    
    temp_test = X_test.sqft_living - tau
    temp_test[temp_test<0] = 0
    
    X_train[f"sqft_living_spline_{i}"] = temp_train**2
    X_val[f"sqft_living_spline_{i}"] = temp_val**2
    X_test[f"sqft_living_spline_{i}"] = temp_test**2

In [229]:
lr.fit(X_train,y_train)

LinearRegression()

In [230]:
predictions = lr.predict(X_test)*sigma_y_train + mu_y_train

In [231]:
lr.score(X_train, y_train)

0.8536899187436561

In [232]:
R2_oos = 1 - np.mean((predictions - y_test)**2)/np.var(y_test)

In [233]:
R2_oos

0.8337806755478809

## Bagging

In [239]:
regr = BaggingRegressor(base_estimator=LinearRegression(), n_estimators=100, random_state=15).fit(X_train, y_train)

In [240]:
regr.score(X_train, y_train)

0.8535919774712412

In [241]:
predictions = regr.predict(X_test)*sigma_y_train + mu_y_train

In [242]:
R2_oos = 1 - np.mean((predictions - y_test)**2)/np.var(y_test)

In [243]:
R2_oos

0.833458507045874

## Boosting 

In [244]:
regr = GradientBoostingRegressor(n_estimators=1000, random_state=15).fit(X_train, y_train)

In [245]:
regr.score(X_train, y_train)

0.974251655589121

In [246]:
predictions = regr.predict(X_test)*sigma_y_train + mu_y_train

In [247]:
R2_oos = 1 - np.mean((predictions - y_test)**2)/np.var(y_test)

In [248]:
R2_oos

0.8814701802524263

## Random Forests

In [254]:
regr = RandomForestRegressor(n_estimators=100, random_state=15).fit(X_train, y_train)

In [255]:
regr.score(X_train, y_train)

0.9815722623087599

In [256]:
predictions = regr.predict(X_test)*sigma_y_train + mu_y_train

In [257]:
R2_oos = 1 - np.mean((predictions - y_test)**2)/np.var(y_test)

In [258]:
R2_oos

0.8474176810965217

## Cross-validation

Until now, to perform model selection, we either used in-sample validation or validation set validation. Both methods, while good starting points, is often less accurate than cross-validation to estimate prediction error and less robust against overfitting.

Cross-validation gives you more stable metrics for how a model is likely to perform because it helps reduce bias. It also helps overcome overfitting and is effective when your (train and test) dataset is small.

In [151]:
kf = KFold(n_splits=10, shuffle=False)

In [152]:
train = pd.read_csv("train.data.csv", low_memory = False, index_col = 0)
test = pd.read_csv("test.data.csv", low_memory = False, index_col = 0)
train.dropna(inplace = True)
test.dropna(inplace = True)
ohe = OneHotEncoder()
categories_train = ohe.fit_transform(train.loc[:, ['zipcode']]).toarray()
categories_test = ohe.transform(test.loc[:, ['zipcode']]).toarray()

In [153]:
for i, cat in enumerate(ohe.categories_[0]):
    train[str(cat)] = categories_train[:, i]
    test[str(cat)] = categories_test[:, i]

In [154]:
train.drop(['zipcode'], axis = 1, inplace = True)
test.drop(['zipcode'], axis = 1, inplace = True)

In [157]:
train.drop(['id', 'date'], axis = 1, inplace = True)
test.drop(['id', 'date'], axis = 1, inplace = True)

In [158]:
full = pd.concat([train, test], axis = 0)

In [173]:
n_estimators = [10*k for k in range(1, 11)]
R2opt = 0
nopt = 0
for n in tqdm(n_estimators):
    score_n = []
    for i, (train_index, test_index) in enumerate(kf.split(full)):
        train, test = full.iloc[train_index, :], full.iloc[test_index, :]
        X_train, y_train, X_test, y_test = train.iloc[:, 1:], train.iloc[:, 0], test.iloc[:, 1:], test.iloc[:, 0]
        regr = BaggingRegressor(base_estimator=LinearRegression(), n_estimators=n, random_state=15).fit(X_train, y_train)
        curr_score = regr.score(X_test, y_test)
        score_n.append(curr_score)
    if np.mean(score_n)>R2opt:
        R2opt = np.mean(score_n)
        nopt = n

100%|██████████| 10/10 [12:39<00:00, 75.99s/it] 


In [174]:
nopt

80

In [175]:
R2opt

0.8059129025560965